<h1>KBG Syndrome</h1>
<p>Data from <a href="https://pubmed.ncbi.nlm.nih.gov/36446582/" target="__blank">Martinez-Cayuelas E, et al. Clinical description, molecular delineation and genotype-phenotype correlation in 340 patients with KBG syndrome: addition of 67 new patients. J Med Genet. 2022 Nov 29:jmedgenet-2022-108632. PMID: 36446582.</a>.</p>

In [1]:
import os
import hpotk

In [2]:
from genophenocorr.preprocessing import configure_caching_patient_creator

In [3]:
fpath_hpo = 'https://github.com/obophenotype/human-phenotype-ontology/releases/download/v2023-10-09/hp.json'
cache_dir = 'annotations'
fpath_phenopackets = 'phenopackets'
tx_id = 'NM_013275.6'


In [4]:
hpo: hpotk.ontology.Ontology = hpotk.ontology.load.obographs.load_ontology(fpath_hpo)

In [5]:
pc = configure_caching_patient_creator(hpo, cache_dir = cache_dir)

In [6]:
from genophenocorr.preprocessing import load_phenopacket_folder

In [7]:
patientCohort = load_phenopacket_folder(fpath_phenopackets, pc)

Patients Created:   0%|                                                                                                                                | 0/328 [00:00<?, ?it/s]Expected a VCF record, a VRS CNV, or an expression with `hgvs.c` but did not find one in patient PMID_36446582_Novara,_2017_P2
Expected at least one variant per patient, but received none for patient PMID_36446582_Novara,_2017_P2
Patient Novara, 2017_P2 has no variants listed and will not be included in this analysis.
Expected a VCF record, a VRS CNV, or an expression with `hgvs.c` but did not find one in patient PMID_36446582_Goldenberg2016_P13
Expected at least one variant per patient, but received none for patient PMID_36446582_Goldenberg2016_P13
Patient Goldenberg2016_P13 has no variants listed and will not be included in this analysis.
Expected a VCF record, a VRS CNV, or an expression with `hgvs.c` but did not find one in patient PMID_36446582_Ockeloen2015_P20
Expected at least one variant per patient, but r

Patient Novara, 2017_P3 has no variants listed and will not be included in this analysis.
Expected a VCF record, a VRS CNV, or an expression with `hgvs.c` but did not find one in patient PMID_36446582_Khalifa,_2013_P1B
Expected at least one variant per patient, but received none for patient PMID_36446582_Khalifa,_2013_P1B
Patient Khalifa, 2013_P1B has no variants listed and will not be included in this analysis.
Expected a VCF record, a VRS CNV, or an expression with `hgvs.c` but did not find one in patient PMID_36446582_Isrie,_2012_P2
Expected at least one variant per patient, but received none for patient PMID_36446582_Isrie,_2012_P2
Patient Isrie, 2012_P2 has no variants listed and will not be included in this analysis.
Expected a VCF record, a VRS CNV, or an expression with `hgvs.c` but did not find one in patient PMID_36446582_Goldenberg2016_P19
Expected at least one variant per patient, but received none for patient PMID_36446582_Goldenberg2016_P19
Patient Goldenberg2016_P19 has 

Patient Goldenberg2016_P4 has no variants listed and will not be included in this analysis.
Expected a VCF record, a VRS CNV, or an expression with `hgvs.c` but did not find one in patient PMID_36446582_Scarano,_2013_P12
Expected at least one variant per patient, but received none for patient PMID_36446582_Scarano,_2013_P12
Patient Scarano, 2013_P12 has no variants listed and will not be included in this analysis.
Expected a VCF record, a VRS CNV, or an expression with `hgvs.c` but did not find one in patient PMID_36446582_Goldenberg2016_P20
Expected at least one variant per patient, but received none for patient PMID_36446582_Goldenberg2016_P20
Patient Goldenberg2016_P20 has no variants listed and will not be included in this analysis.
Expected a VCF record, a VRS CNV, or an expression with `hgvs.c` but did not find one in patient PMID_36446582_Crippa2015_P2
Expected at least one variant per patient, but received none for patient PMID_36446582_Crippa2015_P2
Patient Crippa2015_P2 has n

Expected at least one variant per patient, but received none for patient PMID_36446582_Novara,_2017_P1
Patient Novara, 2017_P1 has no variants listed and will not be included in this analysis.
Expected a VCF record, a VRS CNV, or an expression with `hgvs.c` but did not find one in patient PMID_36446582_Goldenberg2016_P10
Expected at least one variant per patient, but received none for patient PMID_36446582_Goldenberg2016_P10
Patient Goldenberg2016_P10 has no variants listed and will not be included in this analysis.
Expected a VCF record, a VRS CNV, or an expression with `hgvs.c` but did not find one in patient PMID_36446582_Kutkowska-Kazmierczak2021_P21
Expected at least one variant per patient, but received none for patient PMID_36446582_Kutkowska-Kazmierczak2021_P21
Patient Kutkowska-Kazmierczak2021_P21 has no variants listed and will not be included in this analysis.
Expected a VCF record, a VRS CNV, or an expression with `hgvs.c` but did not find one in patient PMID_36446582_Behne

In [8]:
from IPython.display import HTML, display
from genophenocorr.view import CohortViewer

viewer = CohortViewer(hpo)

In [9]:
display(HTML(viewer.cohort_summary_table(patientCohort)))

Item,Description
Total Individuals,251
Total Unique HPO Terms,27
Total Unique Variants,251


In [10]:
display(HTML(viewer.hpo_term_counts_table(patientCohort))) ## Add Labels to output

HPO Term,Count
Macrodontia (HP:0001572),182
Intellectual disability (HP:0001249),159
Abnormality of the hand (HP:0001155),156
Global developmental delay (HP:0001263),133
Short stature (HP:0004322),115
Abnormal external nose morphology (HP:0010938),112
Thick eyebrow (HP:0000574),105
Long philtrum (HP:0000343),103
Hearing impairment (HP:0000365),74
Triangular face (HP:0000325),68


In [11]:
display(HTML(viewer.variants_table(patientCohort, tx_id))) 

Variant,Effect,Count,Key
c.1903_1907del,FRAMESHIFT_VARIANT,33,16_89284634_89284639_GTGTTT_G
c.2408_2412del,FRAMESHIFT_VARIANT,10,16_89284129_89284134_CTTTTT_C
c.1381_1384del,FRAMESHIFT_VARIANT,8,16_89285157_89285161_GTTTC_G
c.2398_2401del,FRAMESHIFT_VARIANT,8,16_89284140_89284144_TTTTC_T
c.6792_6793insC,FRAMESHIFT_VARIANT,5,16_89279749_89279749_C_CG
c.7481_7482insC,FRAMESHIFT_VARIANT,5,16_89275180_89275180_A_AG
c.2182_2183del,FRAMESHIFT_VARIANT,3,16_89284358_89284360_GAT_G
c.3832A>T,STOP_GAINED,3,16_89282710_89282710_T_A
c.2197C>T,STOP_GAINED,3,16_89284345_89284345_G_A
c.7570-1G>C,SPLICE_ACCEPTOR_VARIANT,3,16_89274958_89274958_C_G


In [12]:
from genophenocorr.analysis import configure_cohort_analysis, CohortAnalysisConfiguration
from genophenocorr.analysis.predicate import BooleanPredicate, GroupingPredicate
from genophenocorr.model import VariantEffect

In [13]:
analysis = configure_cohort_analysis(patientCohort, hpo, CohortAnalysisConfiguration(
    missing_implies_excluded = True,
    pval_correction='fdr_bh',
    min_perc_patients_w_hpo=0.1,
    include_sv=True))

In [14]:
frameshift = analysis.compare_by_variant_effect(VariantEffect.FRAMESHIFT_VARIANT, tx_id)

In [15]:
summary_fs = frameshift.summarize(hpo, BooleanPredicate.YES)
summary_fs.sort_values(('','p value'))

FRAMESHIFT_VARIANT on NM_013275.6                  No              Yes  \
                                                Count    Percent Count   
Intellectual disability [HP:0001249]               60  25.210084    99   
Abnormality of mental function [HP:0011446]        66  27.500000   113   
Neurodevelopmental abnormality [HP:0012759]        70  28.925620   127   
Abnormality of the nervous system [HP:0000707]     73  29.918033   135   
Abnormal nervous system physiology [HP:0012638]    73  29.918033   135   
Global developmental delay [HP:0001263]            47  23.039216    86   
Neurodevelopmental delay [HP:0012758]              47  23.039216    86   
Abnormal external nose morphology [HP:0010938]     42  19.811321    70   
Abnormal nasal morphology [HP:0005105]             42  19.811321    70   
Abnormality of the nose [HP:0000366]               42  19.811321    70   
Abnormality of limbs [HP:0040064]                  56  23.236515   100   
Abnormality of the upper limb [HP:0002817]         56  23.236515   100   
Abnormality of the hand [HP:0001155]               56  23.236515   100   
Abnormal lip morphology [HP:0000159]               37  17.129630    66   
Long philtrum [HP:0000343]                         37  17.129630    66   
Abnormality of the philtrum [HP:0000288]           37  17.129630    66   
Abnormal upper lip morphology [HP:0000177]         37  17.129630    66   
Abnormal ear morphology [HP:0031703]               21  10.714286    37   
Abnormality of the orbital region [HP:0000315]     36  17.142857    69   
Abnormality of the integument [HP:0001574]         36  17.142857    69   
Abnormality of the ocular adnexa [HP:0032039]      36  17.142857    69   
Abnormal ocular adnexa morphology [HP:0030669]     36  17.142857    69   
Abnormal hair morphology [HP:0001595]              36  17.142857    69   
Abnormal skin adnexa morphology [HP:0011138]       36  17.142857    69   
Atypical behavior [HP:0000708]                     24  12.631579    48   
Abnormal facial shape [HP:0001999]                 23  11.855670    45   
Triangular face [HP:0000325]                       23  11.855670    45   
Short stature [HP:0004322]                         40  17.467249    75   
Abnormality of body height [HP:0000002]            40  17.467249    75   
Growth delay [HP:0001510]                          40  17.467249    75   
Growth abnormality [HP:0001507]                    40  17.467249    75   
Cognitive impairment [HP:0100543]                  16   8.556150    35   
Phenotypic abnormality [HP:0000118]                80  31.872510   168   
All [HP:0000001]                                   80  31.872510   168   
Abnormality of the ear [HP:0000598]                34  16.267943    77   
Abnormality of head or neck [HP:0000152]           70  28.925620   150   
Abnormality of the face [HP:0000271]               70  28.925620   150   
Abnormality of the head [HP:0000234]               70  28.925620   150   
Abnormality of the mouth [HP:0000153]              65  27.083333   140   
Abnormal oral morphology [HP:0031816]              65  27.083333   140   
Abnormal oral cavity morphology [HP:0000163]       65  27.083333   140   
Abnormal ear physiology [HP:0031704]               22  11.111111    52   
Hearing abnormality [HP:0000364]                   22  11.111111    52   
Abnormal dental morphology [HP:0006482]            58  24.576271   124   
Abnormality of the dentition [HP:0000164]          58  24.576271   124   
Macrodontia [HP:0001572]                           58  24.576271   124   

FRAMESHIFT_VARIANT on NM_013275.6                                     \
                                                   Percent   p value   
Intellectual disability [HP:0001249]             41.596639  0.000305   
Abnormality of mental function [HP:0011446]      47.083333  0.000380   
Neurodevelopmental abnormality [HP:0012759]      52.479339  0.001123   
Abnormality of the nervous system [HP:0000707]   55.327869  0.003385   
Abnormal nervou

In [16]:
var_single = analysis.compare_by_variant_key('16_89284634_89284639_GTGTTT_G')

In [17]:
summary_vs = var_single.summarize(hpo, BooleanPredicate.YES)
summary_vs.sort_values(('','p value'))

>=1 allele of the variant 16_89284634_89284639_GTGTTT_G    No             \
                                                        Count    Percent   
Neurodevelopmental abnormality [HP:0012759]               176  72.727273   
Abnormal nervous system physiology [HP:0012638]           185  75.819672   
Abnormality of the nervous system [HP:0000707]            185  75.819672   
Intellectual disability [HP:0001249]                      144  60.504202   
Phenotypic abnormality [HP:0000118]                       217  86.454183   
All [HP:0000001]                                          217  86.454183   
Abnormality of mental function [HP:0011446]               160  66.666667   
Neurodevelopmental delay [HP:0012758]                     119  58.333333   
Global developmental delay [HP:0001263]                   119  58.333333   
Abnormality of the ear [HP:0000598]                       101  48.325359   
Abnormality of the ocular adnexa [HP:0032039]              95  45.238095   
Abnormality of the orbital region [HP:0000315]             95  45.238095   
Abnormality of the integument [HP:0001574]                 95  45.238095   
Abnormal skin adnexa morphology [HP:0011138]               95  45.238095   
Abnormal hair morphology [HP:0001595]                      95  45.238095   
Abnormal ocular adnexa morphology [HP:0030669]             95  45.238095   
Abnormality of the face [HP:0000271]                      193  79.752066   
Abnormality of the head [HP:0000234]                      193  79.752066   
Abnormality of head or neck [HP:0000152]                  193  79.752066   
Abnormal oral morphology [HP:0031816]                     180  75.000000   
Abnormality of the mouth [HP:0000153]                     180  75.000000   
Abnormal oral cavity morphology [HP:0000163]              180  75.000000   
Abnormal facial shape [HP:0001999]                         62  31.958763   
Triangular face [HP:0000325]                               62  31.958763   
Hearing abnormality [HP:0000364]                           67  33.838384   
Abnormal ear physiology [HP:0031704]                       67  33.838384   
Abnormal upper lip morphology [HP:0000177]                 92  42.592593   
Abnormality of the philtrum [HP:0000288]                   92  42.592593   
Long philtrum [HP:0000343]                                 92  42.592593   
Abnormal lip morphology [HP:0000159]                       92  42.592593   
Abnormality of the nose [HP:0000366]                       99  46.698113   
Abnormal nasal morphology [HP:0005105]                     99  46.698113   
Abnormal external nose morphology [HP:0010938]             99  46.698113   
Abnormal ear morphology [HP:0031703]                       52  26.530612   
Atypical behavior [HP:0000708]                             64  33.684211   
Growth delay [HP:0001510]                                 101  44.104803   
Abnormality of body height [HP:0000002]                   101  44.104803   
Short stature [HP:0004322]                                101  44.104803   
Growth abnormality [HP:0001507]                           101  44.104803   
Abnormality of the dentition [HP:0000164]                 159  67.372881   
Abnormal dental morphology [HP:0006482]                   159  67.372881   
Macrodontia [HP:0001572]                                  159  67.372881   
Abnormality of limbs [HP:0040064]                         134  55.601660   
Abnormality of the upper limb [HP:0002817]                134  55.601660   
Abnormality of the hand [HP:0001155]                      134  55.601660   
Cognitive impairment [HP:0100543]                          44  23.529412   

>=1 allele of the variant 16_89284634_89284639_GTGTTT_G   Yes             \
                                                        Count    Percent   
Neurodevelopmental abnormality [HP:0012759]                21   8.677686   
Abnormal nervous system physiology [HP:0012638]            23   9.426230   
Abnormality of the nervous system [HP:0000707] 

In [18]:
var_double = analysis.compare_by_variant_keys('16_89284129_89284134_CTTTTT_C', '16_89284634_89284639_GTGTTT_G')

In [19]:
summary_vd = var_double.summarize(hpo, BooleanPredicate.YES)
summary_vd.sort_values(('','p value'))

>=1 allele of either variant 16_89284129_89284134_CTTTTT_C or variant 16_89284634_89284639_GTGTTT_G First  \
                                                                                                    Count   
Cognitive impairment [HP:0100543]                                                                       0   
Short stature [HP:0004322]                                                                              5   
Abnormality of body height [HP:0000002]                                                                 5   
Growth delay [HP:0001510]                                                                               5   
Growth abnormality [HP:0001507]                                                                         5   
Hearing abnormality [HP:0000364]                                                                        3   
Abnormal ear physiology [HP:0031704]                                                                    3   
Abnormality of the ear [HP:0000598]                                                                     4   
Abnormality of the hand [HP:0001155]                                                                    8   
Abnormality of the upper limb [HP:0002817]                                                              8   
Abnormality of limbs [HP:0040064]                                                                       8   
Abnormal external nose morphology [HP:0010938]                                                          5   
Abnormal nasal morphology [HP:0005105]                                                                  5   
Abnormality of the nose [HP:0000366]                                                                    5   
Abnormal lip morphology [HP:0000159]                                                                    4   
Long philtrum [HP:0000343]                                                                              4   
Abnormal upper lip morphology [HP:0000177]                                                              4   
Abnormality of the philtrum [HP:0000288]                                                                4   
Abnormal oral cavity morphology [HP:0000163]                                                            8   
Abnormality of the mouth [HP:0000153]                                                                   8   
Abnormality of the dentition [HP:0000164]                                                               8   
Intellectual disability [HP:0001249]                                                                    5   
Abnormality of the head [HP:0000234]                                                                    9   
Abnormality of head or neck [HP:0000152]                                                                9   
Triangular face [HP:0000325]                                                                            2   
Abnormal facial shape [HP:0001999]                                                                      2   
Abnormal dental morphology [HP:0006482]                                                                 8   
Abnormality of the face [HP:0000271]                                                                    9   
Macrodontia [HP:0001572]                                                                                8   
Abnormal ear morphology [HP:0031703]                                                                    1   
Phenotypic abnormality [HP:0000118]                                                                    10   
Abnormality of the nervous system [HP:0000707]                                                          7   
Abnormal nervous system physiology [HP:0012638]                                                         7   
Neurodevelopmental abnormality [HP:0012759]                                                             6   
Abnormality of mental function [HP:0011446]                                 